<a href="https://colab.research.google.com/github/DaoLua/Opella./blob/Data_Logger/Data_Logger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tabula
import jpype
import pandas as pd
import re
import os

# --- 1. SETUP DIRECTORY AND LIST PDF FILES ---
datalogger_dir = '/content/datalogger/'
os.makedirs(datalogger_dir, exist_ok=True)
pdf_files = [f for f in os.listdir(datalogger_dir) if f.endswith('.pdf')]

all_processed_dataframes = []
total_tor_data = []

# --- GET USER INPUT FOR GLOBAL FILTERING ---
print("\nStep 1: Thời gian nhập nguyên liệu vào kho")
while True:
    try:
        # Get date and time from user in YYYY/MM/DD HH:MM format
        input_date_str = input("Nhập ngày (YYYY/MM/DD): ")
        input_time_str = input("Nhập giờ (HH:MM): ")

        # Combine and convert to a datetime object
        cutoff_str = f"{input_date_str} {input_time_str}:00"
        global_cutoff_datetime = pd.to_datetime(cutoff_str, format='%Y/%m/%d %H:%M:%S')
        break
    except ValueError:
        print("Sai format. Thử lại.")

# --- GET USER INPUT FOR TOR TEMPERATURE THRESHOLD ---
print("\nStep 2: Nhiệt độ giới hạn của nguyên liệu")
while True:
    try:
        tor_threshold_temp = float(input("Nhập nhiệt độ (°C): "))
        break
    except ValueError:
        print("Sai rồi. Thử lại.")


# --- 3. PROCESS EACH PDF FILE ---
for pdf_file in pdf_files:
    pdf_path = os.path.join(datalogger_dir, pdf_file)
    logger_id = pdf_file

    print(f"\nĐang xử lý file: {logger_id}")

    # --- EXTRACT TABLES FROM PDF ---
    try:
        dfs = tabula.read_pdf(pdf_path, pages="2-5", stream=True, multiple_tables=True)
    except Exception as e:
        print(f"Lỗi đọc file PDF {pdf_file}: {e}")
        continue

    # --- RESHAPE DATA FROM WIDE TO LONG FORMAT ---
    all_reshaped_chunks = []
    for page_df in dfs:
        for i in range(0, page_df.shape[1], 3):
            if i + 2 < page_df.shape[1]:
                chunk = page_df.iloc[:, i:i+3].copy()
                chunk.columns = ['raw_col1', 'raw_col2', 'raw_col3']
                all_reshaped_chunks.append(chunk)
    if not all_reshaped_chunks:
        # print(f"No data chunks found in {pdf_file}. Skipping.")
        continue
    raw_long_df = pd.concat(all_reshaped_chunks, ignore_index=True)

    # --- CLEAN THE RESHAPED DATA ---
    all_records = []
    pattern = re.compile(r'(\d{2}/\d{2}/\d{2})?\s+(\d{2}:\d{2}:\d{2})\s+([\d\.\-]+)')

    combined_series = raw_long_df['raw_col1'].fillna('') + ' ' + \
                      raw_long_df['raw_col2'].fillna('') + ' ' + \
                      raw_long_df['raw_col3'].fillna('')

    for text in combined_series:
        matches = pattern.findall(text)
        for date_val, time_val, temp_val in matches:
            all_records.append({
                'date': date_val,
                'time': time_val,
                'temperature': temp_val})

    if not all_records:
        # print(f"No records found in {pdf_file}. Skipping.")
        continue

    # --- CREATE THE DATAFRAME ---
    datalogger = pd.DataFrame(all_records)
    datalogger['datetime'] = pd.to_datetime(datalogger['date'] + ' ' + datalogger['time'], format='%m/%d/%y %H:%M:%S', errors='coerce')
    datalogger['temperature'] = pd.to_numeric(datalogger['temperature'], errors='coerce')
    datalogger['temperature'] = datalogger['temperature'].astype('float16')
    datalogger.drop(columns=['date', 'time'], inplace=True)

    datalogger.dropna(subset=['datetime', 'temperature'], inplace=True)
    datalogger.sort_values('datetime', inplace=True)
    datalogger.drop_duplicates(subset=['datetime', 'temperature'], keep='first', inplace=True)
    datalogger = datalogger[['datetime', 'temperature']].reset_index(drop=True)

    if datalogger.empty:
        # print(f"No valid data remaining in {pdf_file} after cleaning. Skipping.")
        continue

    # --- FILTER THE DATAFRAME USING GLOBAL CUTOFF ---
    # print(f"\nStep 4: Filtering data to remove records after {global_cutoff_datetime} for {logger_id}...")

    # Keep only the rows where the datetime is less than or equal to the user's input
    datalogger_filtered = datalogger[datalogger['datetime'] <= global_cutoff_datetime].copy()

    # print(f"Filtering complete for {logger_id}. {len(datalogger_filtered)} records remaining.")

    if datalogger_filtered.empty:
        # print(f"No data remaining after filtering for {logger_id}. Skipping TOR analysis.")
        continue

    # --- Calculate Min and Max Temperature and their datetimes for the filtered data ---
    min_temp = datalogger_filtered['temperature'].min()
    max_temp = datalogger_filtered['temperature'].max()
    min_temp_datetime = datalogger_filtered[datalogger_filtered['temperature'] == min_temp]['datetime'].min()
    max_temp_datetime = datalogger_filtered[datalogger_filtered['temperature'] == max_temp]['datetime'].min()


    # --- TOR ANALYSIS ---
    print(f"\nStep 5: Thực hiện TOR analysis cho Logger ID {logger_id} với giới hạn nhiệt độ {tor_threshold_temp}°C...")
    datalogger_filtered['is_tor'] = datalogger_filtered['temperature'] > tor_threshold_temp

    tor_periods = []
    start_time = None

    for i in range(len(datalogger_filtered)):
        is_current_tor = datalogger_filtered.loc[i, 'is_tor']
        is_previous_tor = False if i == 0 else datalogger_filtered.loc[i-1, 'is_tor']
        is_next_tor = False if i == len(datalogger_filtered) - 1 else datalogger_filtered.loc[i+1, 'is_tor']

        if is_current_tor and not is_previous_tor:
            start_time = datalogger_filtered.loc[i, 'datetime']

        if is_current_tor and not is_next_tor and start_time is not None:
            end_time = datalogger_filtered.loc[i, 'datetime']
            tor_periods.append({'TOR start': start_time, 'TOR stop': end_time})
            start_time = None

    tor_df = pd.DataFrame(tor_periods)

    total_tor_file = pd.Timedelta(seconds=0)
    if not tor_df.empty:
        tor_df['Excursion duration'] = tor_df['TOR stop'] - tor_df['TOR start']

        # Format Excursion duration
        tor_df['Excursion duration'] = tor_df['Excursion duration'].apply(lambda x: f'{x.days*24 + x.seconds//3600:02d}:{x.seconds%3600//60:02d}')

        # Add temperature at TOR start and stop
        tor_df = pd.merge(tor_df, datalogger_filtered[['datetime', 'temperature']],
                                left_on='TOR start', right_on='datetime', how='left')
        tor_df.rename(columns={'temperature': 'temperature at tor start'}, inplace=True)
        tor_df.drop(columns=['datetime'], inplace=True)

        tor_df = pd.merge(tor_df, datalogger_filtered[['datetime', 'temperature']],
                                left_on='TOR stop', right_on='datetime', how='left')
        tor_df.rename(columns={'temperature': 'temperature at tor stop'}, inplace=True)
        tor_df.drop(columns=['datetime'], inplace=True)

        # Rearrange columns
        final_tor_df = tor_df[['TOR start', 'temperature at tor start', 'TOR stop', 'temperature at tor stop', 'Excursion duration']]

        # Add Logger ID
        final_tor_df['Logger ID'] = logger_id

        all_processed_dataframes.append(final_tor_df)

        # Calculate total TOR for this file (in timedelta)
        tor_df['Excursion duration_timedelta'] = tor_df['Excursion duration'].apply(lambda x: pd.to_timedelta(x + ':00'))
        total_tor_file = tor_df['Excursion duration_timedelta'].sum()


    else:
        # print(f"No TOR periods found in {pdf_file} after filtering.")

    # Store total TOR, min temp, max temp, and their datetimes for this logger
    total_tor_data.append({
        'Logger ID': logger_id,
        'Total TOR': total_tor_file,
        'Min Temperature': min_temp,
        'Min Temp Datetime': min_temp_datetime,
        'Max Temperature': max_temp,
        'Max Temp Datetime': max_temp_datetime
    })


# --- 4. COMBINE DATAFRAMES ---
if all_processed_dataframes:
    combined_datalogger_df = pd.concat(all_processed_dataframes, ignore_index=True)
    combined_datalogger_df = combined_datalogger_df.reset_index(drop=True)

    # --- 5. CALCULATE AND FORMAT TOTAL TOR PER LOGGER ---
    total_tor_summary_df = pd.DataFrame(total_tor_data)

    def format_timedelta_as_ddhhmm(td):
        """Formats a timedelta object into 'dd:hh:mm' string."""
        total_seconds = td.total_seconds()
        days = int(total_seconds // (24 * 3600))
        hours = int((total_seconds % (24 * 3600)) // 3600)
        minutes = int((total_seconds % 3600) // 60)
        return f"{days:02d}:{hours:02d}:{minutes:02d}"

    # Format the total TOR
    total_tor_summary_df['Total TOR'] = total_tor_summary_df['Total TOR'].apply(format_timedelta_as_ddhhmm)


    # --- 6. FINALIZE AND DISPLAY RESULTS ---
    # Add a sequential number column for each Logger ID in the combined_datalogger_df
    combined_datalogger_df['Number'] = combined_datalogger_df.groupby('Logger ID').cumcount() + 1

    # Format temperature columns to 2 decimal places in the combined_datalogger_df
    combined_datalogger_df['temperature at tor start'] = combined_datalogger_df['temperature at tor start'].round(2)
    combined_datalogger_df['temperature at tor stop'] = combined_datalogger_df['temperature at tor stop'].round(2)

    # Format min/max temperature columns to 2 decimal places in the summary df
    total_tor_summary_df['Min Temperature'] = total_tor_summary_df['Min Temperature'].round(2)
    total_tor_summary_df['Max Temperature'] = total_tor_summary_df['Max Temperature'].round(2)


    # Rearrange columns to have 'Logger ID' first, then 'Number'
    final_combined_df = combined_datalogger_df[['Logger ID', 'Number', 'TOR start', 'temperature at tor start', 'TOR stop', 'temperature at tor stop', 'Excursion duration']]

    # print("--- Combined TOR Excursion Details with Logger ID, Number, and Temperatures ---")
    display(final_combined_df)

    print("\n--- Total Time Out of Refrigerator (TOR) per Logger ID, Min/Max Temperature, and Datetime ---")
    # Rearrange columns for summary display
    total_tor_summary_df = total_tor_summary_df[['Logger ID', 'Total TOR', 'Min Temperature', 'Min Temp Datetime', 'Max Temperature', 'Max Temp Datetime']]
    display(total_tor_summary_df)

    # --- 7. EXPORT TO EXCEL ---
    output_excel_path = "/content/datalogger/tor_analysis_results.xlsx"
    with pd.ExcelWriter(output_excel_path) as writer:
        final_combined_df.to_excel(writer, sheet_name='Excursion Details', index=False)
        total_tor_summary_df.to_excel(writer, sheet_name='Total TOR Summary', index=False)

    print(f"\nResults exported successfully to '{output_excel_path}'")


else:
    print("No data processed from any PDF files.")